In [1]:
pip install dataset accelerate databench_eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import subprocess
import shlex
import zipfile
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from databench_eval import Runner, Evaluator

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b")
# model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-3b",torch_dtype="auto",)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-5.7bmqa-base")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-5.7bmqa-base",torch_dtype="auto",)
model.cuda()

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=128, bias=False)
          (v_proj): Linear(in_features=4096, out_features=128, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm

In [4]:
def call_model(prompts):
  """
  tokenize prompt, model generate;
  """
  results = []
  for p in prompts:
      inputs = tokenizer(p, return_tensors="pt").to(model.device)
      tokens = model.generate(
          **inputs,
          max_new_tokens=128,
          temperature=0.2,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
      result = tokenizer.decode(tokens[0], skip_special_tokens=True)
      results.append(result)
  return results

In [5]:
# Load the main QA data from three .txt files into a DataFrame
qa_df = pd.DataFrame()

# Read the answers from the .txt files into separate lists
with open("/kaggle/input/answers/answers.txt", "r") as f:
    answers = f.read().splitlines()

with open("/kaggle/input/answers/answers_lite.txt", "r") as f:
    sample_answers = f.read().splitlines()

with open("/kaggle/input/answers/semantics.txt", "r") as f:
    semantics = f.read().splitlines()

# Combine the lists into a DataFrame

# Load the dataset column from the specified file
qa_df = pd.read_csv("/kaggle/input/competition/competition/test_qa.csv")
qa_df["answer"] = answers
qa_df["sample_answer"] = sample_answers
qa_df["type"] = semantics


# Convert to Dataset
qa = Dataset.from_pandas(qa_df)
evaluator = Evaluator(qa=qa)
# load data and evaluator
def load_table(dataset):
    return pd.read_parquet(f"/kaggle/input/competition/competition/{dataset}/all.parquet")

def load_sample(dataset):
    return pd.read_parquet(f"/kaggle/input/competition/competition/{dataset}/sample.parquet")
# evaluator = Evaluator(qa=qa)

In [6]:
qa[0]

{'question': 'Is our average employee older than 35?',
 'dataset': '066_IBM_HR',
 'answer': 'True',
 'sample_answer': 'True',
 'type': 'boolean'}

In [7]:
err_dict,none_str_dict = {},{}
for i in range(len(qa)):
   for value_ in qa[i].values():
    if isinstance(value_, str):
      if value_.startswith(" "):
        err_dict[i]=qa[i]
    else:
      none_str_dict[i]=qa[i]
err_dict

{14: {'question': ' Which gender is most satisfied with the job on average?',
  'dataset': '066_IBM_HR',
  'answer': 'Male',
  'sample_answer': 'Male',
  'type': 'category'},
 95: {'question': ' How much were the 4 largest contracts in central and west africa awarded for? Answer with a list with one amount per contract',
  'dataset': '068_WorldBank_Awards',
  'answer': '[321127567.87, 306306411.37, 253905847.27, 180832613.5]',
  'sample_answer': '[716716.34, 369866.76, 198952.48, 189088.46]',
  'type': 'list[number]'},
 143: {'question': ' Are more than a 1000 products sold in Mercadona stores?',
  'dataset': '070_OpenFoodFacts',
  'answer': 'True',
  'sample_answer': 'False',
  'type': 'boolean'},
 158: {'question': ' List the product codes associated with the Seitan category',
  'dataset': '070_OpenFoodFacts',
  'answer': '[00001522, 8480000041968, 8480000041975, 8480000046444, 8480000230058, 8480000865274, 8480000707000]',
  'sample_answer': '[00001522]',
  'type': 'list[number]'},


In [8]:
def custom_compare(value, truth, semantic):
  # override the default compare method;
    semantic = semantic.strip()  # add this line to remove the space;

    if semantic == "boolean":
        return str(value) == str(truth)
    elif semantic == "category":
        return str(value) == str(truth)
    elif semantic == "number":
        try:
            value_cleaned = ''.join(char for char in str(value) if char.isdigit() or char in ['.', '-'])
            truth_cleaned = ''.join(char for char in str(truth) if char.isdigit() or char in ['.', '-'])
            return round(float(value_cleaned), 2) == round(float(truth_cleaned), 2)
        except :
            return False
    elif semantic == "list[category]":
        try:
            value_list = [item.strip() for item in str(value).strip('[]').split(',')]
            truth_list = [item.strip() for item in str(truth).strip('[]').split(',')]
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    elif semantic == "list[number]":
        try:
            value_list = sorted(round(float(''.join(c for c in v.strip() if c.isdigit() or c in ['.', '-'])), 2) for v in str(value).strip('[]').split(',') if v.strip())
            truth_list = sorted(round(float(''.join(c for c in t.strip() if c.isdigit() or c in ['.', '-'])), 2) for t in str(truth).strip('[]').split(',') if t.strip())
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    else:
        raise ValueError(f"Semantic not supported: '{semantic}'")

In [9]:
custom_evaluator = Evaluator(compare=custom_compare,qa=qa)

In [10]:
prompts = [] # prompt
code_res_full = [] # generated code, full text
code_res_run = [] # generated code, trunct till the first \n, what is actually running

In [11]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}
    return"""

    prompts.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full = response.split("return")[2]
        resp_run = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full.append(resp_full) # append to code_responses
        code_res_run.append(resp_run)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [12]:
runner = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, load_table
    ),
    qa=qa,
    batch_size=10,
)

In [13]:
responses = runner.run(save="predictions.txt")

  0%|          | 0/53 [00:00<?, ?it/s]<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  2%|▏         | 1/53 [01:07<58:40, 67.71s/it]<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current beha

In [14]:
print(f"DataBench accuracy is {evaluator.eval(responses)}")
# custom_evaluator.eval(responses)

100%|██████████| 522/522 [00:00<00:00, 1215.29it/s]

DataBench accuracy is 0.4904214559386973


In [15]:
# df_responses = {
#     # 'prompt': prompts,
#     # 'full_code_response': code_res_full,
#     # "code_implement":code_res_run,
#     'data_ans': responses,
#     # "gold": qa["answer"]
# }

# df = pd.DataFrame(df_responses)
# df

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')
# df.to_csv('/kaggle/working/response_stable-code-3b.csv', index=False)
# df.to_csv('/kaggle/working/predictions.txt', index=False)

In [17]:
prompts_lite = [] # prompt
code_res_full_lite = [] # generated code, full text
code_res_run_lite = [] # generated code which is actually running

In [18]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}
    return"""

    prompts_lite.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full_lite = response.split("return")[2]
        resp_run_lite = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full_lite.append(resp_full_lite)
        code_res_run_lite.append(resp_run_lite)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [19]:
runner_lite = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, load_sample
    ),
    qa=qa,
    batch_size=10,
)

In [20]:
responses_lite = runner_lite.run(save="predictions_lite.txt")

  0%|          | 0/53 [00:00<?, ?it/s]<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position

In [21]:
print(f"DataBench_lite accuracy is {custom_evaluator.eval(responses_lite, lite=True)}")

100%|██████████| 522/522 [00:00<00:00, 58877.71it/s]

DataBench_lite accuracy is 0.4942528735632184


In [22]:
# df_responses_lite = {
#     # 'prompt': prompts_lite,
#     # 'full_code_response': code_res_full_lite,
#     # "running_code_response":code_res_run_lite,
#     'data_ans': responses_lite,
#     # "gold": qa["answer"]
# }

# df_lite = pd.DataFrame(df_responses_lite)
# df_lite

In [23]:
# from google.colab import drive
# drive.mount('/content/drive')
# df_lite.to_csv('/kaggle/working/response_lite_stable-code-3b.csv', index=False)
# df.to_csv('/kaggle/working/predictions_lite.txt', index=False)

In [24]:
# with zipfile.ZipFile("submissions_TableGPT2-7B.zip", "w") as zipf:
#     zipf.write("predictions.txt")
#     zipf.write("predictions_lite.txt")

# print("Created results_CodeLlama_7b.zip.zip containing predictions.txt and predictions_lite.txt")